In [1]:
import sys

sys.path.insert(0, '../bifacial_radiance')

try:
    from bifacial_radiance import *

except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')
    # Simple example system using Radiance.

In [10]:
# The below routine creates a HEXAGONAL torque tube, for a 2-UP configuration of a specific module size. Parameters for the module, the torque tube, and the scene are below.
# This is being run with gendaylit, for one specific timestamp

timestamp = 4020 # Noon, June 17th. 
simulationname = 'Torque_tube_hex_test'

# MakeModule Parameters
module_type='my_custom_panel'
x = 1.996  # 2-up portrait Longi with 15cm additional gap
y = 0.991
orientation='portrait' 
tilt = 10

# TorqueTube Parameters
torqueTube = True
numpanels = 2
gap = 0.15
tubetype = 'Hex'
disttopanel = 0.175
diameter = 0.15
torqueTubeMaterial = 'black'       # IT's NOT GRAY, IT's GREY

# SceneDict Parameters
gcr = 0.33   # ground cover ratio,  = module_height / pitch
albedo = 0.28  #'concrete'     # ground albedo
clearance_height = 2.35   
azimuth_ang=90
sensorsy = 200 # Notice we will be doing 200 sensors! It takes more time but gives better resolution.
psx = 0.01

# Cell level module
cellLevelModule=True
numcellsx=6
numcellsy=10
xcell=0.156
ycell=0.156
xgap=0.02
ygap=0.02


In [3]:
# MODIFY THE TEST FOLDER TO YOUR LIKING. I utmostly dislike the interactive directory.
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Demo2'

In [11]:
# Now let's run the example

demo = RadianceObj('Multicell',path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(0.62) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
epwfile = demo.getEPW(37.5,-77.6) # pull TMY data for any global lat/lon
metdata = demo.readEPW(epwfile) # read in the EPW weather data from above
demo.gendaylit(metdata,4020)  # Noon, June 17th

# Making module with all the variables
moduledict=demo.makeModule(name=module_type,x=x,y=y,bifi=1, 
           torquetube=torqueTube, diameter = diameter, tubetype = tubetype, material = torqueTubeMaterial, tubeZgap = disttopanel, numpanels = numpanels, panelgap = gap, rewriteModulefile = True, psx=psx,
                          cellLevelModule=cellLevelModule, numcellsx=numcellsx, numcellsy=numcellsy, xcell=xcell, ycell=ycell, xgap=xgap, ygap=ygap)
# create a scene with all the variables
sceneDict = {'tilt':tilt,'pitch': np.round(moduledict['sceney'] / gcr,3),'height':clearance_height,'azimuth':azimuth_ang, 'module_type':module_type}  

scene = demo.makeScene(moduletype=module_type, sceneDict=sceneDict, nMods = 3, nRows = 1, sensorsy = sensorsy) #makeScene creates a .rad file with 20 modules per row, 7 rows.
octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.


path = C:\Users\sayala\Documents\RadianceScenes\Demo2
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw  ... OK!
('\nModule Name:', 'my_custom_panel')
REWRITING pre-existing module file. 
Module was shifted by 0.078 in X to avoid sensors on air
This is a Cell-Level detailed module with Packaging Factor of 0.81 %
Module my_custom_panel successfully created
Created Multicell.oct


Now you can do Sanity check with rvu. It takes A WHILE since it is generating so many surfaces

rvu -vf views\front.vp -e .01 Multicell.oct

In [9]:
analysis = AnalysisObj(octfile, demo.name)  # return an analysis object including the scan dimensions for back irradiance
analysis.analysis(octfile, demo.name, scene.frontscan, scene.backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio average:  %0.3f' %( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) ) )


Linescan in process: Multicell_Front
Linescan in process: Multicell_Back
Saved: results\irr_Multicell.csv
Annual bifacial ratio average:  0.517


In [ ]:
(numcellsx-1) % 2